<a href="https://colab.research.google.com/github/ElomKossi/Traffic_Signs_Recogniction/blob/main/train_ts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train traffic signs dataset with Yolo

**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Dec  9 07:05:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'	    'Secret Story'	 UTBM_Printemps_2019
 machine-learning-training  'Stage ST40'	 UTBM_Printemps_2020
'My Drive'		     UTBM_Automne_2018	 yoloV3
 Reglement.pdf		     UTBM_Automne_2020


**1) Compile Darknet using Nvidia GPU**

In [ ]:
%cd /content/gdrive/MyDrive/yoloV3/darknet

/content/gdrive/MyDrive/yoloV3/darknet


In [ ]:
%cd /content/gdrive/MyDrive/yoloV3/darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/gdrive/MyDrive/yoloV3/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src

**2) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/content/gdrive/MyDrive/yoloV3"

mkdir: cannot create directory ‘/content/gdrive/MyDrive/yoloV3’: File exists


In [ ]:
!echo "ts_data" > data/ts_data.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/ts_data.names\nbackup = /mydrive/yoloV3' > data/ts_data.data
!rm -rf data/ts_data
!mkdir data/ts_data

In [ ]:
!unzip /content/gdrive/MyDrive/yoloV3/ts_data.zip -d data

Archive:  /content/gdrive/MyDrive/yoloV3/ts_data.zip
  inflating: data/ts_data/ts_data.data  
replace data/__MACOSX/ts_data/._ts_data.data? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/__MACOSX/ts_data/._ts_data.data  
  inflating: data/ts_data/classes.txt  
replace data/__MACOSX/ts_data/._classes.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/__MACOSX/ts_data/._classes.txt  
  inflating: data/ts_data/.DS_Store  
replace data/__MACOSX/ts_data/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/__MACOSX/ts_data/._.DS_Store  
  inflating: data/ts_data/train.txt  
replace data/__MACOSX/ts_data/._train.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/__MACOSX/ts_data/._train.txt  
  inflating: data/ts_data/ts_data.names  
replace data/__MACOSX/ts_data/._ts_data.names? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/__MACOSX/ts_data/._ts_data.names  
  inflating: data/ts_data/test.txt   
replace data/__MACOSX/ts_data/._

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/ts_data/ts/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(numbers[0], numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
8 0.3522058823529412 0.6575 0.03529411764705882 0.06

['8', '0.3522058823529412', '0.6575', '0.03529411764705882', '0.06']
130 data/ts_data/ts/00390.txt
8 0.3522058823529412 0.6575 0.03529411764705882 0.06
8 0.913235294117647 0.646875 0.033823529411764704 0.05875

['8', '0.913235294117647', '0.646875', '0.033823529411764704', '0.05875']
130 data/ts_data/ts/00390.txt
8 0.913235294117647 0.646875 0.033823529411764704 0.05875
10 0.6235294117647059 0.7075 0.01764705882352941 0.03

['10', '0.6235294117647059', '0.7075', '0.01764705882352941', '0.03']
131 data/ts_data/ts/00353.txt
10 0.6235294117647059 0.7075 0.01764705882352941 0.03
10 0.34301470588235294 0.7125 0.015441176470588236 0.0275

['10', '0.34301470588235294', '0.7125', '0.015441176470588236', '0.0275']
131 data/ts_data/ts/00353.txt
10 0.34301470588235294 0.7125 0.015441176470588236 0.0275
30 0.6922794117647059 0.71375 0.04191176470588235 0.065

['30', '

In [ ]:
import glob
images_list = glob.glob("data/ts_data/ts/*.jpg")
print(images_list)

['data/ts_data/ts/00612.jpg', 'data/ts_data/ts/00809.jpg', 'data/ts_data/ts/00404.jpg', 'data/ts_data/ts/00376.jpg', 'data/ts_data/ts/00638.jpg', 'data/ts_data/ts/00610.jpg', 'data/ts_data/ts/00410.jpg', 'data/ts_data/ts/00835.jpg', 'data/ts_data/ts/00606.jpg', 'data/ts_data/ts/00362.jpg', 'data/ts_data/ts/00565.jpg', 'data/ts_data/ts/00766.jpg', 'data/ts_data/ts/00000.jpg', 'data/ts_data/ts/00571.jpg', 'data/ts_data/ts/00772.jpg', 'data/ts_data/ts/00564.jpg', 'data/ts_data/ts/00217.jpg', 'data/ts_data/ts/00014.jpg', 'data/ts_data/ts/00015.jpg', 'data/ts_data/ts/00203.jpg', 'data/ts_data/ts/00798.jpg', 'data/ts_data/ts/00202.jpg', 'data/ts_data/ts/00001.jpg', 'data/ts_data/ts/00029.jpg', 'data/ts_data/ts/00773.jpg', 'data/ts_data/ts/00028.jpg', 'data/ts_data/ts/00388.jpg', 'data/ts_data/ts/00363.jpg', 'data/ts_data/ts/00824.jpg', 'data/ts_data/ts/00165.jpg', 'data/ts_data/ts/00820.jpg', 'data/ts_data/ts/00405.jpg', 'data/ts_data/ts/00159.jpg', 'data/ts_data/ts/00171.jpg', 'data/ts_data

In [ ]:
#Create training.txt file
file = open("data/ts_data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close()

**3) Start the training**

In [ ]:
# Running training process in Darknet framework
! ./darknet detector train cfg/ts_data.data cfg/yolov3_ts_train.cfg weights/darknet53.conv.74 -dont_show

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000002, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.637846, GIOU: 0.587100), Class: 0.554361, Obj: 0.405569, No Obj: 0.000118, .5R: 0.666667, .75R: 0.333333, count: 3, class_loss = 1.299269, iou_loss = 0.722892, total_loss = 2.022161 
 total_bbox = 102255, rewritten_bbox = 0.126155 % 

 2121: 0.803780, 0.629662 avg loss, 0.001000 rate, 4.921851 seconds, 67872 images, 166.748431 hours left
Loaded: 0.000070 seconds
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000003, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, 

In [6]:
# Continue training with saved weights after XXXX iterations
! ./darknet detector train cfg/ts_data.data cfg/yolov3_ts_train.cfg backup/yolov3_ts_train_last.weights -dont_show

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.717170, GIOU: 0.696518), Class: 0.995233, Obj: 0.982751, No Obj: 0.000302, .5R: 1.000000, .75R: 0.333333, count: 3, class_loss = 0.145645, iou_loss = 0.264601, total_loss = 0.410246 
 total_bbox = 130745, rewritten_bbox = 0.137673 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000001, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000320, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.413128, iou_loss = 0.000000, total_loss = 0.413128 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.772223, GIOU: 0.759164), Class: 0.85

In [13]:
# Calculating mAP for Traffic Signs dataset
!./darknet detector map cfg/ts_data.data cfg/yolov3_ts_train.cfg backup/yolov3_ts_train_last.weights

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 